# 0.INPUTS (`l_country_data`, `l_coverage`, `l_stores`, `l_states`)

### PATHS

In [1]:
import os
os.chdir('/Code/ditto_v2')
%pwd

'c:\\Code\\ditto_v2'

In [2]:
from src.functionality_maps import f_maps, Defs
import geopandas as gpd

### VARIABLES

In [3]:
gadm = 'D2'
l_country_data = [
    # 'D1',
    'D2','D3', 'D4', 'ZIP', #'census'
    # gadm, 
    'census'
    ]
l_coverage = '20'

# l_stores = ['sportscheck', 'decathlon','intersport','sport2000','sportxx','karstadt','engelhorn','sportarena','sportcheck','sportsohle]
# l_stores = ['sportscheck', 'decathlon','intersport']
l_stores = ['decathlon']

d_inputs = {
    'stores' : {
        'sportscheck' : {
            'color' : 'orange'
        },
        'decathlon' : {
            'color' : (0.0, 130.0, 195.0, 255.0) # 'darkblue' # 
        },
        'intersport' : {
            'color' : 'darkred'
        }
    }
}
# l_stores = ['decathlon']

# l_states = ['Bayern','Baden-Württemberg','Hessen','Nordrhein-Westfalen','Rheinland-Pfalz', \
#             'Saarland','Sachsen','Sachsen-Anhalt','Thüringen','Berlin','Brandenburg','Mecklenburg-Vorpommern', \
#                 'Hamburg','Niedersachsen','Schleswig-Holstein','Bremen']
l_states = ['Baden-Württemberg']#, 'Bayern']

is_logging = True
round_dec = 2
is_simplified = False
simplification_tolerance = .1 #(0.01 unit in projected CRS means 0.01 unit in reality)

# 1.INGESTION (`d_ger`, `radius_km`)

### LOAD DATA (`d_ger`, `radius_km`)

In [4]:
########## LOAD 'GERMANY' `d_ger` ##############
import pandas as pd

pd.set_option('display.max_columns', None)
d_ger = f_maps.load_germany(l_levels=l_country_data,
                            is_logging=True)

#### PARSING VARIABLES ###
radius_km = int(l_coverage)

# d_ger['census'].head(1)
# d_ger['census'].head(1)

Level [D2-State] loaded!
 No columns to normalize
Level [D3-Region] loaded!
 No columns to normalize


Level [D4-District] loaded!
 No columns to normalize
Level [ZIP-Locality] loaded!
 No columns to normalize
Census is loaded!
 Normalizing non-serializable columns... ['BEGINN', 'WSK']
 Columns ['BEGINN', 'WSK'] removed


In [5]:
print(d_ger['D2'].shape)
print(d_ger['D3'].shape)
print(d_ger['D4'].shape)
print(d_ger['ZIP'].shape)
print(d_ger['census'].shape)

(16, 4)
(40, 12)
(434, 14)
(8700, 3)
(10990, 28)


# 2.CLEAN/FILTER (`pdf_stores`, `gpd_census`)
[`pdf_stores`: with Store data

`gpd_census`: with Population data
]

In [6]:
from src.functionality_maps import f_maps, paths, Defs
import pandas as pd
import geopandas as gpd

### PDF_STORES ###
print('Filtering stores...')
pdf_stores = pd.read_json(paths.stores, orient='records', lines=True)
print(f'  Stores [{l_stores}]... # stores:{pdf_stores.shape[0]}')
pdf_stores = pdf_stores[pdf_stores['store'].isin(l_stores)]
print(f'  After filtering stores [{l_stores}]... # stores:{pdf_stores.shape[0]}')
pdf_stores = pdf_stores[pdf_stores['GADM_1'].isin(l_states)]
print(f'  After filtering states [{l_states}]... # stores:{pdf_stores.shape[0]}\n')
print(f'  Stores {list(pdf_stores['name'])}')

## ADD NAME OF STORE
pdf_stores['store_name'] = pdf_stores["store"] +"_" +  pdf_stores["name"]

#### EXTRACTING LOCATIONS FROM STORES ####
l_center_points = list(zip(pdf_stores['lat'], pdf_stores['lon'], pdf_stores['store_name']))

Filtering stores...
  Stores [['decathlon']]... # stores:508
  After filtering stores [['decathlon']]... # stores:89
  After filtering states [['Baden-Württemberg']]... # stores:14

  Stores ['Karlsruhe-Durlach', 'Ulm', 'Plochingen', 'Baden - Baden', 'Aalen-Essingen', 'Schwetzingen', 'Böblingen', 'Lörrach', 'Ludwigsburg', 'Heilbronn-K3', 'Offenburg', 'Singen', 'Stuttgart-Milaneo', 'Waiblingen']


In [7]:

l_bundeslaender = [Defs.dict_bundeslaender_id[x] for x in l_states]

#### FILTERING GEOMETRY #####
filter_entity = paths.d_map_gadm_name[gadm]
print(f'Filtering by GADM ({gadm}/{filter_entity} in list {l_states})')

gpd_geom = d_ger['D2'] ## ALWAYS USE TOWNS/CITIES because they contain all info for above divisions
print(f'  # States:{gpd_geom.shape[0]}')
gpd_geom = gpd_geom[gpd_geom['name'].isin(l_states)]
print(f'  After filtering, # States:{gpd_geom.shape[0]}')

#### FILTERING CENSUS #####
print('Filtering census...')
gdf_census = d_ger['census']
print(f'  # Towns:{gdf_census.shape[0]}')
gdf_census = gdf_census[gdf_census['SN_L'].isin(l_bundeslaender)]
print(f'  After filtering, # Towns:{gdf_census.shape[0]}')

Filtering by GADM (D2/State in list ['Baden-Württemberg'])
  # States:16
  After filtering, # States:1
Filtering census...
  # Towns:10990
  After filtering, # Towns:1103


In [8]:
def merge_census_geom(gdf:gpd.GeoDataFrame, 
                      col_groupby:str = 'SN_L', col_geom:str='geometry',
                      is_logging:bool=False) -> gpd.GeoDataFrame:
    if is_logging:
        print(f'Grouping geometries...')
    gdf = gdf.groupby('SN_L').agg( \
        {'EWZ':'sum', 
        'KFL' : 'sum',
        'geometry': lambda x: x.geometry.union_all(),
        })
    if is_logging:
        print(f' Adding areas and renaming columns...')

    gdf = gdf.reset_index(names=['Bundesland'])
    gdf = gpd.GeoDataFrame(gdf).set_crs(4326)
    gdf['Bundesland'] = gdf['Bundesland'].map(Defs.dict_id_bundeslaender)
    gdf['area_geom'] = gdf.to_crs(6933).geometry.area/ 10**6
    gdf['area_geom'] = gdf['area_geom'].round(round_dec)
    gdf = gdf.rename(columns={'KFL':'area_dbms'})
    return gdf



path_census_D2 = './assets/Geo/Germany/D1_Deutschland_census.json'
# gdf_census_merged = merge_census_geom(d_ger['census'], is_logging=is_logging)
# gdf_census_merged.to_file(path_census_D2)
gdf_census_merged = gpd.read_file(path_census_D2)
gdf_census_merged

,Bundesland,EWZ,area_dbms,area_geom,geometry
0,Schleswig-Holstein,2953270,15254,15624.87,"MULTIPOLYGON (((8.93016 53.91648, 8.9309 53.91..."
1,Hamburg,1892122,755,742.55,"MULTIPOLYGON (((9.75824 53.55644, 9.75984 53.5..."
2,Niedersachsen,8140242,47241,47661.54,"MULTIPOLYGON (((7.23817 52.264, 7.23681 52.264..."
3,Bremen,684864,419,402.48,"MULTIPOLYGON (((8.51135 53.22795, 8.51264 53.2..."
4,Nordrhein-Westfalen,18139116,33921,34110.72,"MULTIPOLYGON (((6.38501 50.38419, 6.38684 50.3..."
5,Hessen,6391360,20895,21114.92,"MULTIPOLYGON (((8.37035 49.86278, 8.36813 49.8..."
6,Rheinland-Pfalz,4159150,18715,19856.54,"POLYGON ((6.41666 49.54395, 6.41432 49.54326, ..."
7,Baden-Württemberg,11280257,35219,35748.88,"MULTIPOLYGON (((7.71191 47.5388, 7.71067 47.53..."
8,Bayern,13369393,69430,70544.94,"POLYGON ((9.6502 47.55793, 9.6492 47.55843, 9...."
9,Saarland,992666,2545,2571.02,"POLYGON ((6.83629 49.21126, 6.8371 49.2108, 6...."


In [9]:
gdf_census_merged = gdf_census_merged[gdf_census_merged['Bundesland'].isin(l_states)]
gdf_census_merged

,Bundesland,EWZ,area_dbms,area_geom,geometry
7,Baden-Württemberg,11280257,35219,35748.88,"MULTIPOLYGON (((7.71191 47.5388, 7.71067 47.53..."


In [10]:
if is_simplified:
    gdf_census['geometry'] = gdf_census.geometry.simplify(simplification_tolerance)
    gdf_census_merged['geometry'] = gdf_census_merged.geometry.simplify(simplification_tolerance)

# gdf_census_simp.info()
# gdf_census_merged.info()

# 3.PROCESSING (`gdf_geom_6933`, `gdf_geom_4326`)

## 3.0. Adding lat lon to `gdf_census`

In [11]:
gdf_geom_4326 = gdf_census.to_crs(epsg=4326)

#### COMPUTING AREAS ####
print('Computing areas...')
gdf_census_6993 = gdf_census.to_crs(epsg=6933)
gdf_census_6993['KFL_GPD'] = round(gdf_census_6993.geometry.area / 10**6, 2)
gdf_census_6993['area_geom'] = round(gdf_census_6993['geometry'].area / 10**6, 2)

gdf_census_6993['centroid']=gdf_census_6993.to_crs('+proj=cea').centroid.to_crs(gdf_census_6993.crs)
gdf_census_6993['centroid']= gdf_census_6993['centroid'].to_crs(epsg=4326)
gdf_census_6993['lat']=gdf_census_6993['centroid'].y
gdf_census_6993['lon']=gdf_census_6993['centroid'].x

gdf_census_6993 = gdf_census_6993.drop(['centroid'], axis=1)
gdf_census_6993.head(1)

Computing areas...


,OBJID,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,NBD,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,EWZ,KFL,DLM_ID,EPK,EPK_norm,geometry,KFL_GPD,area_geom,lat,lon
5198,DEBKGVG20000040K,6,4,1,081110000000,08111000,081110000000,Stuttgart,Stadt,60,kreisfrei,ja,08,1,11,00,00,000,R,DE111,081110000000,08111000,632865,207,DEBKGDL20000DZKQ,3057.318841,0.0,"POLYGON ((890102.993 5519238.077, 890086.186 5...",209.95,209.95,48.774528,9.172003


## 3.1. GEOMETRIES `fglc_geom` (Region, Towns)

### Feature Group Geometries (`fg_geom_region` using `gdf_census_merged`)

In [12]:
import folium

fg_geom_pop = round(gdf_census_merged.EWZ.sum() / 10**6, round_dec)
fg_geom_area = gdf_census_merged.area_geom.sum()

# Area_dbms:{gdf_census_merged.area_dbms.sum()}Km2
# fg_legend = f'Geometry Region of Interest [States: {l_states}, \
#         Pop:{fg_geom_pop}M, \
#         Area_geom:{fg_geom_area}Km2]'
fg_legend = f'Region(s) of Interest [#={len(l_states)}, Pop:{fg_geom_pop}M, Area:{fg_geom_area}Km2]'

fg_geom_region = folium.FeatureGroup(name=fg_legend)
gjson_store = f_maps.get_folium_geojson( gdf_census_merged, 
                                fields=['Bundesland', 'EWZ', 'area_geom'],
                                aliases = ['Bundesland', 'Pop', 'Area_geom[Km2]']
                                )
gjson_store.add_to(fg_geom_region)

### Feature Group Geometries (`fg_geom_town` using `gdf_census`)

In [13]:
num_towns = gdf_census.shape[0]
fg_geom_pop = round(gdf_census.EWZ.sum() / 10**6, round_dec)
fg_geom_area = gdf_census_6993.area_geom.sum()

# legend_geom_town = f'Towns in RoI [#Towns: {num_towns}, \
#         Pop:{fg_geom_pop}M, \
#         Area_geom:{fg_geom_area}Km2]'
legend_geom_town = f'Towns in RoI [#={num_towns}]'


fg_geom_town = folium.FeatureGroup(name=legend_geom_town)
gjson_store = f_maps.get_folium_geojson( gdf_census_6993.to_crs(4326), 
                                fields=['GEN', 'EWZ', 'area_geom', 'KFL'],
                                aliases = ['Town', 'Pop', 'Area_geom[km2]', 'Area_dbms[Km2]']
                                )
gjson_store.add_to(fg_geom_town)

In [14]:
l_fg_geometry = [fg_geom_region, fg_geom_town]
fglc_geom = folium.plugins.GroupedLayerControl(
    groups={'Geometry': l_fg_geometry},
    exclusive_groups=False,
    collapsed=False,
)

## 3.2. STORES (Markers, Circles)

In [15]:
if ('centroid' in gdf_census_6993.columns):
    gdf_census_6993 = gdf_census_6993.drop(columns=['centroid'])

col_perc_dbms = 'PERC_dbms_int'
col_perc_geom = 'PERC_geom_int'
col_perc_pop = 'PERC_pop'
l_cols_percentaged = ['area_geom', 'EWZ', 'KFL']

d_columns = {
    'area_geom':{'alias' : 'area',        'is_norm': True,  'is_int':False},
    'KFL':     {'alias' : 'area_data',    'is_norm': True,  'is_int':False},
    'EWZ':     {'alias' : 'Population',   'is_norm': True,  'is_int':True}
}

In [16]:
from src.functionality_maps import f_maps, f_gadm

d_fglc = {}
# l_stores= ['sportscheck']
l_stores= ['decathlon']
# l_stores = ['sportscheck', 'decathlon','intersport']
for i,store_sel in enumerate(l_stores):
    print(f'[{i+1}/{len(l_stores)}] Store={store_sel}')

    ### FILTER BY STORE ###
    pdf_stores_store = pdf_stores[pdf_stores['store'] == store_sel]
    print(f'  # stores={pdf_stores_store.shape[0]}')

    ### 1. FG WITH MARKERS ###
    print(f' ### 1. FG WITH MARKERS ###')
    fg_mk_store = f_gadm.get_fg_store(pdf_stores_store, [store_sel])
    fg_mk_store = fg_mk_store[0]
    # fg_stores = f_maps.get_fg_markers(df=pdf_stores, 
    #                                   name=f'Stores [{pdf_stores.shape[0]}]',
    #                                   l_tooltip=['name', 'address'],
    #                                   l_popup=['url'])

    ### 2. FG WITH CIRCLES AROUND STORES ###
    print(f' ### 2. FG WITH CIRCLES AROUND STORES ###')
    gdf_circles_store = f_maps.get_gdf_circle(df=pdf_stores_store, radius_km= radius_km)
    fg_circ_store = folium.FeatureGroup(name=f'Circles [r={radius_km}Km]')
    f_maps.get_folium_geojson(gdf_circles_store, 
                                fields=['store','name', 'address'],
                                aliases = ['store', 'name', 'address'],
                                fill_color = d_inputs['stores'][store_sel]['color']
                                ).add_to(fg_circ_store)

    ### 3. FG WITH JOIN OF GEOM AND CIRCLES ###
    print('### 3. FG WITH JOIN OF GEOM AND CIRCLES ###')
    gdf_joi_circ_geom_store = gpd.sjoin(gdf_geom_4326, gdf_circles_store.to_crs(4326), how="inner", predicate="intersects")
    gdf_joi_circ_geom_store['KFL_GPD'] = round(gdf_census_6993.to_crs(6933).geometry.area / 10**6, round_dec)

    ### FEATURE GROUP ###
    fg_joi_circ_geom_store = folium.FeatureGroup(name=f'Join')
    gjson_store = f_maps.get_folium_geojson(gdf_joi_circ_geom_store, 
                                    fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD'],
                                    aliases = ['Name', 'Population', 'Area_dbms', 'Area_Geom']).add_to(fg_joi_circ_geom_store)


    ### 4. FG WITH INTERSECTION OF GEOM AND CIRCLES ###
    print(f" ### 4. FG WITH INTERSECTION OF GEOM AND CIRCLES ###")
    gdf_int_circ_geom = gpd.overlay(gdf_census.to_crs(epsg=6933), 
                                    gdf_circles_store[['geometry','store','store_name']].to_crs(6933), 
                                    how='intersection')
    gdf_int_circ_geom['area'] = gdf_int_circ_geom.geometry.area / 10**6

    fg_int_circ_geom_store = folium.FeatureGroup('Intersection')
    f_maps.get_folium_geojson(gdf_int_circ_geom,#.to_crs(4326), 
                                fields=['GEN','store_name', 'EWZ', 'KFL', 'area'],
                                aliases = ['Location','store_name', 'Pop', 'Area_dbms[Km2]', 'Area_geom[Km2]'],
                                fill_color = d_inputs['stores'][store_sel]['color']).add_to(fg_int_circ_geom_store)
    


    # #### ADDED FOR PERCENTAGE INTERSECTION
    # ########################################
    # ########################################

    fg_join_ind = folium.FeatureGroup(name=f'Individual Join')

    # Convert the Shapely circle to a geoDataFrame
    poly_circle = f_maps.circle_latlon(lon=lon, lat=lat, radius=1000 * radius_km)
    gdf = gpd.GeoSeries([poly_circle])
    gdf_overlay = gdf.to_frame(name='geometry').set_crs(epsg=4326)
    gdf_circle_6933 = gdf_overlay.to_crs(epsg=6933)
    gdf_overlay = gpd.overlay(gdf_census_6993, gdf_circle_6933, how='intersection')

    # Set the column used for NORMALIZATION (numerator)
    gdf_overlay['area'] = round(gdf_overlay['geometry'].area / 10**6, 3)
    gdf_overlay[col_perc_geom] = round(gdf_overlay['area'] / gdf_overlay['area_geom'], 3)
    
    gdf_overlay = f_maps.overlay_shapes(gdf_circle=gdf_overlay,
                                        col_perc=col_perc_geom, 
                                        d_columns=d_columns)
    gdf_overlay = gdf_overlay.drop_duplicates(subset=['OBJID'])
    gdf_overlay[col_perc_dbms] = round(gdf_overlay['KFL_int'] / gdf_overlay['KFL'], 3)
    gdf_overlay[col_perc_pop] = round(gdf_overlay['EWZ_int'] / gdf_overlay['EWZ'], 3)
    gdf_overlay['store_name'] = store_name

    gjson_store = f_maps.get_folium_geojson(gdf_overlay,#.to_crs(4326), 
                                    fields=[
                                        'store_name', 'GEN',
                                        # 'KFL_int', 'KFL', # col_perc_dbms, 
                                        'area_geom_int', 'area_geom', col_perc_geom,
                                        'EWZ_int', 'EWZ', col_perc_pop,
                                        # 'num_in', 'num_part', 
                                        ],
                                    aliases = [
                                        'Store', 'City',
                                        # 'Area_dbms intersection [Km2]', 'Area_dbms shape [Km2]','Percentage dbms', 
                                        'Area_geom intersection [Km2]', 'Area_geom shape [Km2]', 'Percentage geometry',
                                        'Population inside', 'Population total', 'Percentage population'
                                        # 'Localities inside', 'Localities partially inside',
                                        ])
    gjson_store.add_to(fg_join_ind)

#     # #### ADDED FOR PERCENTAGE INTERSECTION
#     # ########################################
#     # ########################################


    


    ### 5. FG WITH MERGE OF GEOM AND CIRCLES ###
    print(f" ### 5. FG WITH MERGE OF GEOM AND CIRCLES ###")
    gdf_mer_circ_geom = gdf_int_circ_geom.groupby(['store','store_name']).agg( \
        {'geometry': lambda x: x.geometry.union_all(),
        'EWZ':'sum', 
        'KFL' : 'sum',
        }).set_geometry("geometry").set_crs(6933).reset_index()
    gdf_mer_circ_geom['area_geom'] = gdf_mer_circ_geom.geometry.area / 10**6
    fg_mer_circ_geom_store = folium.FeatureGroup('Merge')
    gjson_store = f_maps.get_folium_geojson(gdf_mer_circ_geom.to_crs(4326),
                            fields=['store', 'store_name', 'EWZ', 'area_geom', 'KFL'],
                            aliases = ['store','store_name', 'Pop', 'Area_geom[Km2]', 'Area_dbms[Km2]'],
                            fill_color = d_inputs['stores'][store_sel]['color']).add_to(fg_mer_circ_geom_store)

    ### 6. FG WITH UNION OF ALL (GEOM,CIRCLES) ###
    print(f" ### 6. FG WITH UNION OF ALL (GEOM,CIRCLES) ###")
    poly_merged = gdf_mer_circ_geom.geometry.union_all()
    gdf_store_circles_merged = gpd.GeoSeries([poly_merged]).to_frame(name='geometry').set_crs(epsg=6933)
    num_stores = gdf_mer_circ_geom.shape[0]
    area_total = round(gdf_store_circles_merged.to_crs(6933).geometry.area.sum() / 10**6, round_dec)
    # perc_area  = round(area_total/fg_geom_area, round_dec)



    gdf_int_merg_store = gpd.overlay(gdf_census_6993, gdf_store_circles_merged.to_crs(6933), how='intersection')
    Pop_sum=round(gdf_int_merg_store.EWZ.sum()/10**6 , round_dec)
    print(f'Pop sum={Pop_sum}M')
    # str_legend = f'[# Stores={num_stores},' \
    #             f'area ={area_total}Km2, ' \
    #             f'perc_area = {perc_area}]'

    gdf_store_circles_merged['num_stores'] = num_stores
    gdf_store_circles_merged['KFL'] = gdf_mer_circ_geom.KFL.sum()
    gdf_store_circles_merged['EWZ'] = Pop_sum
    gdf_store_circles_merged['area'] = round(gdf_store_circles_merged.geometry.area.sum()/10**6, round_dec)
    gdf_store_circles_merged['perc_area'] = round(gdf_store_circles_merged['area'] / fg_geom_area, round_dec)
    gdf_store_circles_merged['perc_pop'] = round((Pop_sum) / fg_geom_pop, round_dec)
    print(f'Total pop={fg_geom_pop}')

    fg_uni_circ_geom_store = folium.FeatureGroup(name=f'Union')
    gjson_store = f_maps.get_folium_geojson(gdf_store_circles_merged.to_crs(4326),
                                    fields=['num_stores','area', 'perc_area', 'EWZ', 'perc_pop'],
                                    aliases = ['# Stores','Area [Km2]', 'Area perc.', 'Pop', 'Pop perc.'],
                                    is_styled = True,
                                    fill_color = d_inputs['stores'][store_sel]['color']
                                    ).add_to(fg_uni_circ_geom_store)






    ### ADDING all FG into a list ###
    print(' Adding all FGs')
    l_fg_store = [                  # For a STORE CHAING (e.g. Intersport)
        fg_mk_store,                    # FG with IND Markers of stores
        fg_circ_store,                  # FG with IND GeoJsons of circles of radius `radius_km` around stores
        fg_joi_circ_geom_store,         # FG with IND GeoJsons of the JOIN (Geometries, circles around stores)
        fg_int_circ_geom_store,         # FG with IND GeoJsons of the INTERSECTION (Geometries, circles around stores)
        # fg_merge_int,
        fg_mer_circ_geom_store,         # FG with IND GeoJsons of the MERGE (Geometries, circles around stores)
        fg_uni_circ_geom_store          # FG with ALL GeoJsons of the UNION of the MERGE (Geometries, circles around stores)
        ]

    ### GENERATING GroupedLayerControl FOR STORE ###
    fglc_store = folium.plugins.GroupedLayerControl(
        groups={f'{store_sel}': l_fg_store},
        exclusive_groups=False,
        collapsed=False,
    )

    ### SAVING IT IN THE DICTIONARY ###
    d_fglc[store_sel] = {
        'l_children' : l_fg_store,
        'fglc' : fglc_store,
        'fg_1_mk_stores' : fg_mk_store,
        'fg_2_gj_circles': fg_circ_store,
        'fg_3_gj_join': fg_joi_circ_geom_store,
        'fg_4_gj_intersect': fg_int_circ_geom_store,
        'fg_5_gj_merge' : fg_mer_circ_geom_store,
        'fg_6_gj_union' : fg_uni_circ_geom_store,

        'df_5_merge' : gdf_mer_circ_geom,
        'df_6_union' : gdf_store_circles_merged
    }

[1/1] Store=decathlon
  # stores=14
 ### 1. FG WITH MARKERS ###


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Code\ditto_v2\src\functionality_maps\f_gadm.py:193: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projected_points = [transform(proj_wgs84, proj_utm, lon, lat) for lat, lon in points_area]
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.g

 ### 2. FG WITH CIRCLES AROUND STORES ###
### 3. FG WITH JOIN OF GEOM AND CIRCLES ###
 ### 4. FG WITH INTERSECTION OF GEOM AND CIRCLES ###


NameError: name 'lon' is not defined

## 3.3. ALL STORES `fg_merged_allstores`

In [47]:
df = pd.DataFrame()
for i,(k,v) in enumerate(d_fglc.items()):
    print(f'[{i+1}/{len(d_fglc)}] Store:{k}')
    # v["key"] = k
    df = pd.concat([df, v['df_6_union']], ignore_index=True)
df['BL'] = 'BW'
df

[1/1] Store:decathlon


,geometry,num_stores,KFL,EWZ,area,perc_area,perc_pop,BL
0,"MULTIPOLYGON (((832564.14 5415358.655, 832571....",14,21308,7745903,11707.24,0.33,686693.53,BW


In [ ]:
poly_merged_allstores = df.geometry.union_all()
gdf_merged_allstores = gpd.GeoSeries([poly_merged_allstores]).to_frame(name='geometry').set_crs(epsg=6933)

gdf_merged_allstores = df.groupby('BL').agg( \
        {'geometry': lambda x: x.geometry.union_all(),
        'num_stores':'sum', 
        }).set_geometry("geometry").set_crs(6933).reset_index()

gdf_merged_allstores_int_census = gpd.overlay(gdf_census_6993, 
                                              gdf_merged_allstores.to_crs(6933), how='intersection')

Pop_sum = gdf_merged_allstores_int_census.EWZ.sum()
gdf_merged_allstores['EWZ'] = Pop_sum
gdf_merged_allstores['area'] = round(gdf_merged_allstores.geometry.area.sum()/10**6, round_dec)
gdf_merged_allstores['perc_area'] = round(gdf_merged_allstores['area'] / fg_geom_area, round_dec)
gdf_merged_allstores['perc_pop'] = round((Pop_sum/10**6) / fg_geom_pop, round_dec)

# gdf_merged_allstores

fg_merged_allstores = folium.FeatureGroup(name=f'Stores[{gdf_merged_allstores.num_stores.values[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_merged_allstores.to_crs(4326),
                                fields=['num_stores','area', 'perc_area', 'EWZ', 'perc_pop'],
                                aliases = ['# Stores','Area [Km2]', 'Area perc.', 'Pop', 'Pop perc.'],
                                is_styled = True
                                ).add_to(fg_merged_allstores)

l_fg_merged_allstores = [fg_merged_allstores]
### GENERATING GroupedLayerControl FOR STORE ###
fglc_all_stores = folium.plugins.GroupedLayerControl(
    groups={'All stores': l_fg_merged_allstores},
    exclusive_groups=False,
    collapsed=False,
)

## 3.4. NEW CODE -> GEOM+STORES (Intersections,Merges)

### Feature group Intersection (`fg_int_circ_geom` using `gdf_int_circ_geom`)

In [35]:
gdf_int_circ_geom = gpd.overlay(gdf_census.to_crs(epsg=6933), 
                                gdf_store_circles[['geometry','store','store_name']].to_crs(6933), 
                                how='intersection')

In [41]:
fg_int_circ_geom = folium.FeatureGroup('overlay')
gjson_store = f_maps.get_folium_geojson(gdf_int_circ_geom,#.to_crs(4326), 
                        fields=['GEN','store_name', 'EWZ', 'KFL'],
                        aliases = ['Location','store_name', 'Pop', 'Area_dbms'])
gjson_store.add_to(fg_int_circ_geom)

### Feature group Merge (`fg_mer_circ_geom` using `gdf_mer_circ_geom`)

In [37]:
gdf_mer_circ_geom = gdf_int_circ_geom.groupby(['store','store_name']).agg( \
        {'geometry': lambda x: x.geometry.union_all(),
        'EWZ':'sum', 
        'KFL' : 'sum',
        }).set_geometry("geometry").set_crs(6933).reset_index()
gdf_mer_circ_geom['area_geom'] = gdf_mer_circ_geom.geometry.area / 10**6
# gdf_me.head()

In [38]:
fg_mer_circ_geom = folium.FeatureGroup('merge')
gjson_store = f_maps.get_folium_geojson(gdf_mer_circ_geom.to_crs(4326),
                        fields=['store', 'store_name', 'EWZ', 'area_geom', 'KFL'],
                        aliases = ['store','store_name', 'Pop', 'Area_geom[Km2]', 'Area_dbms[Km2]'])
gjson_store.add_to(fg_mer_circ_geom)

## 3.5.OLD CODE

### Feature Group Circles (`fg_circ` using `gdf_store_circles`)
[`pdf_stores` and `radius_km`]

In [29]:
import math, folium

gdf_store_circles = f_maps.get_gdf_circle(df=pdf_stores, radius_km= radius_km)
# gdf_store_circles.shape

fg_geom_area = gdf_census_merged.area_geom.sum()
num_stores = gdf_store_circles.shape[0]
area_circ  = round(math.pi * radius_km**2, round_dec)
area_total = round(num_stores*area_circ, round_dec)
perc_area  = round(area_total/fg_geom_area, round_dec)

str_legend = f'[# Circles={num_stores}, radius={radius_km}Km, ' \
             f'area circle={area_circ}Km2, ' \
             f'area total ={area_total}Km2, ' \
             f'perc_area = {perc_area}]'
# str_legend
fg_circ_store = folium.FeatureGroup(name=f'Circles {str_legend}')
gjson_store = f_maps.get_folium_geojson(gdf_store_circles, 
                                fields=['store','name', 'address'],
                                # aliases = ['Shop']
                                )
gjson_store.add_to(fg_circ_store)

<!-- ### Featour Group Circles union (`fg_circ_union`) -->

### Feature Group Circles (`fg_circ_merged` using `gdf_store_circles_merged`)
[`pdf_stores` and `radius_km`])

In [30]:
poly_merged = gdf_store_circles.geometry.union_all()
gdf_store_circles_merged = gpd.GeoSeries([poly_merged]).to_frame(name='geometry').set_crs(epsg=4326)
num_stores = gdf_store_circles.shape[0]
area_total = round(gdf_store_circles_merged.to_crs(6933).geometry.area.sum() / 10**6, round_dec)
perc_area  = round(area_total/fg_geom_area, round_dec)


str_legend = f'[# Stores={num_stores},' \
             f'area ={area_total}Km2, ' \
             f'perc_area = {perc_area}]'

gdf_store_circles_merged['num_stores'] = num_stores
gdf_store_circles_merged['area'] = area_total
gdf_store_circles_merged['perc_area'] = perc_area


fg_circ_merged = folium.FeatureGroup(name=f'Circles Merged {str_legend}')
gjson_store = f_maps.get_folium_geojson(gdf_store_circles_merged, 
                                fields=['num_stores','area', 'perc_area'],
                                aliases = ['# Stores','Area [Km2]', 'Percentage'],
                                is_styled = True
                                )
gjson_store.add_to(fg_circ_merged)

### Feature Group MERGE (`fg_merge` using [`gdf_intersection`, `gdf_overlay`])

In [34]:
if ('centroid' in gdf_census_6993.columns):
    gdf_census_6993 = gdf_census_6993.drop(columns=['centroid'])

col_perc_dbms = 'PERC_dbms_int'
col_perc_geom = 'PERC_geom_int'
col_perc_pop = 'PERC_pop'
l_cols_percentaged = ['area_geom', 'EWZ', 'KFL']

d_columns = {
    'area_geom':{'alias' : 'area',        'is_norm': True,  'is_int':False},
    'KFL':     {'alias' : 'area_data',    'is_norm': True,  'is_int':False},
    'EWZ':     {'alias' : 'Population',   'is_norm': True,  'is_int':True}
}

In [35]:
############# OVERLAY / individual ####################

# Set the column used for NORMALIZATION (denominator)
gdf_intersection = gpd.overlay(gdf_census_6993, gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_intersection = gdf_intersection.drop_duplicates()
gdf_intersection['area'] = round(gdf_intersection['geometry'].area / 10**6, 2)
gdf_intersection[col_perc_geom] = gdf_intersection['area'] / gdf_intersection['area_geom']

# Generate Overlay with the intersection and drop duplicates
gdf_overlay = f_maps.overlay_shapes(gdf_circle=gdf_intersection,
                                    col_perc=col_perc_geom, 
                                    d_columns=d_columns)
gdf_overlay_dd = gdf_overlay.drop_duplicates(subset=['OBJID'])

# Merge Overlay
gdf_merge = f_maps.merge_shapes(gdf_intersection, l_col_percs=l_cols_percentaged,
                                d_percs={'data':'KFL', 'geom':'area_geom'}, 
                                num_dec=round_dec, is_logging=True)
gdf_merge = gdf_merge.drop_duplicates()


total_area_geom = round(gdf_census_6993.geometry.area.sum() / 10**6, round_dec)
total_area_pop = round(gdf_census_6993.EWZ.sum() / 10**6, round_dec)
# merge_area_dbms = round(gdf_census_6993.KFL.sum(), round_dec)
# merge_area_circ = round(gdf_store_circles_merged.to_crs(6933).geometry.area.sum() / 10**6, round_dec)
merge_area_inside = round(gdf_intersection["area"].sum() , round_dec)
perc_area_inside  = round(merge_area_inside/total_area_geom, round_dec)
pop_inside =  round(gdf_merge.EWZ_int.sum() / 10**6, round_dec)
pop_total =  round(gdf_merge.EWZ.sum() / 10**6, round_dec)
perc_pop = round(pop_inside/total_area_pop, round_dec)


gdf_merge['EWZ'] = gdf_merge.EWZ.sum()
gdf_merge['KFL'] = gdf_merge.KFL.sum()

# area_dbms={merge_area_dbms}Km2,  \
# area_circ_merged={merge_area_circ}Km2, \
msg_merge = f'area_int={merge_area_inside}Km2, \
              total_area_geom={total_area_geom}Km2,  \
              perc_area={perc_area_inside}, \
              Pop_inside={pop_inside}M,  \
              Pop_total={total_area_pop}M, \
              perc_pop={perc_pop} '


fg_merge = folium.FeatureGroup(name=f'Merge (Geom, Circles) [{msg_merge}]')

# gdf_merge['num_in'] = gdf_inside.shape[0]
# gdf_merge['num_part'] = gdf_partly.shape[0]
gjson_store = f_maps.get_folium_geojson(gdf_merge, 
                                  fields=[
                                    # 'KFL_int', 'KFL', 'perc_data', 
                                    'area_geom_int', 'area_geom', 'perc_geom',
                                    'EWZ_int', 'EWZ',
                                    # 'num_in', 'num_part', 
                                    ],
                                  aliases = [
                                    # 'Area_dbms intersection [Km2]', 'Area_dbms shape [Km2]','Percentage dbms', 
                                    'Area_geom intersection [Km2]', 'Area_geom shape [Km2]','Percentage geometry',
                                    'Population intersection', 'Population shape',
                                    # '# Localities inside', 
                                    # '# Localities partially inside',
                                    ])
gjson_store.add_to(fg_merge)

area_geom = 15824.6
area_geom_int = 11707.2
EWZ = 7745903
EWZ_int = 6464690
KFL = 15571
KFL_int = 11501.14
perc_data = 0.74
perc_geom = 0.74


### Feature Group INDIV (`fg_join_ind` and `fg_merge_ind`)

In [32]:
fg_join_ind = folium.FeatureGroup(name=f'Individual Join [#Stores={len(l_center_points)}]')
fg_merge_ind = folium.FeatureGroup(name=f'Individual Merge [#Stores={len(l_center_points)}]')
# gdf_sc_6933 = gdf_store_circles.to_crs(6933)

# for i,(index,store) in enumerate(gdf_sc_6933.iterrows()):
#     print(f'[{i}/{gdf_store_circles.shape[0]}] Store={store.store}_{store['name']}')

for i,center_point in enumerate(l_center_points):
    print(f'[{i+1}/{len(l_center_points)}] center_point={center_point}')
    lon = center_point[1]
    lat = center_point[0]
    store_name = center_point[2]

    # Convert the Shapely circle to a geoDataFrame
    poly_circle = f_maps.circle_latlon(lon=lon, lat=lat, radius=1000 * radius_km)
    gdf = gpd.GeoSeries([poly_circle])
    gdf_overlay = gdf.to_frame(name='geometry').set_crs(epsg=4326)
    gdf_circle_6933 = gdf_overlay.to_crs(epsg=6933)
    gdf_overlay = gpd.overlay(gdf_census_6993, gdf_circle_6933, how='intersection')

    # Set the column used for NORMALIZATION (numerator)
    gdf_overlay['area'] = round(gdf_overlay['geometry'].area / 10**6, 3)
    gdf_overlay[col_perc_geom] = round(gdf_overlay['area'] / gdf_overlay['area_geom'], 3)
    
    gdf_overlay = f_maps.overlay_shapes(gdf_circle=gdf_overlay,
                                        col_perc=col_perc_geom, 
                                        d_columns=d_columns)
    gdf_overlay = gdf_overlay.drop_duplicates(subset=['OBJID'])
    gdf_overlay[col_perc_dbms] = round(gdf_overlay['KFL_int'] / gdf_overlay['KFL'], 3)
    gdf_overlay[col_perc_pop] = round(gdf_overlay['EWZ_int'] / gdf_overlay['EWZ'], 3)
    gdf_overlay['store_name'] = store_name

    gjson_store = f_maps.get_folium_geojson(gdf_overlay,#.to_crs(4326), 
                                    fields=[
                                        'store_name', 'GEN',
                                        # 'KFL_int', 'KFL', # col_perc_dbms, 
                                        'area_geom_int', 'area_geom', col_perc_geom,
                                        'EWZ_int', 'EWZ', col_perc_pop,
                                        # 'num_in', 'num_part', 
                                        ],
                                    aliases = [
                                        'Store', 'City',
                                        # 'Area_dbms intersection [Km2]', 'Area_dbms shape [Km2]','Percentage dbms', 
                                        'Area_geom intersection [Km2]', 'Area_geom shape [Km2]', 'Percentage geometry',
                                        'Population inside', 'Population total', 'Percentage population'
                                        # 'Localities inside', 'Localities partially inside',
                                        ])
    gjson_store.add_to(fg_join_ind)

    gdf_merge = f_maps.merge_shapes(gdf_overlay, l_col_percs=l_cols_percentaged,
                                            d_percs={'data':'KFL', 'geom':'area_geom'}, 
                                            num_dec=round_dec, is_logging=False)
    # Copying Location Identifier
    gdf_merge['GEN'] = gdf_overlay['GEN']
    # Set the column used for NORMALIZATION (numerator)
    gdf_merge['area'] = round(gdf_merge['geometry'].area / 10**6, round_dec)
    gdf_merge[col_perc_geom] = round(gdf_merge['area'] / gdf_merge['area_geom'], 3)
    gdf_merge[col_perc_dbms] = round(gdf_merge['KFL_int'] / gdf_merge['KFL'], 3)
    gdf_merge[col_perc_pop] = round(gdf_merge['EWZ_int'] / gdf_merge['EWZ'], 3)
    gdf_merge['store_name'] = store_name
    
    gjson_store = f_maps.get_folium_geojson(gdf_merge,#.to_crs(4326), 
                            fields=[
                                'store_name',
                                # 'KFL_int', 'KFL', col_perc_dbms,
                                'area_geom_int', 'area_geom', col_perc_geom,
                                'EWZ_int', 'EWZ', col_perc_pop,
                                # 'num_in', 'num_part', 
                                ],
                            aliases = [
                                'Location',
                                # 'Area_dbms intersection [Km2]', 'Area_dbms shape [Km2]', 'Percentage dbms', 
                                'Area_geom intersection [Km2]', 'Area_geom shape [Km2]', 'Percentage area',
                                'Population intersection', 'Population shape', 'Percentage population'
                                # 'Localities inside', 'Localities partially inside',
                                ])
    gjson_store.add_to(fg_merge_ind)

    if i==0:
        gdf_merge_concat = gdf_merge
    else:
        gdf_merge_concat = pd.concat([gdf_merge_concat, gdf_merge], axis=0)

[1/14] center_point=(49.00488565, 8.449674128073184, 'decathlon_Karlsruhe-Durlach')
[2/14] center_point=(48.4007813, 9.9583729, 'decathlon_Ulm')
[3/14] center_point=(48.7026898, 9.437392619846309, 'decathlon_Plochingen')
[4/14] center_point=(48.7965766, 8.19518157114592, 'decathlon_Baden - Baden')
[5/14] center_point=(48.8201891, 10.045626348192611, 'decathlon_Aalen-Essingen')
[6/14] center_point=(49.3941626, 8.57783239839136, 'decathlon_Schwetzingen')
[7/14] center_point=(48.6882907, 9.0090554, 'decathlon_Böblingen')
[8/14] center_point=(47.610891699999996, 7.661843945356753, 'decathlon_Lörrach')
[9/14] center_point=(48.900043, 9.1916945, 'decathlon_Ludwigsburg')
[10/14] center_point=(49.1460258, 9.2214379, 'decathlon_Heilbronn-K3')
[11/14] center_point=(48.472267200000005, 7.944426365060242, 'decathlon_Offenburg')
[12/14] center_point=(47.7599556, 8.8420214, 'decathlon_Singen')
[13/14] center_point=(48.791258850000006, 9.182104646022687, 'decathlon_Stuttgart-Milaneo')
[14/14] center_

### `fg_join` EXTERNAL

In [50]:
# gdf_geom_4326.head(1)

############# JOIN ####################
gdf_join = gpd.sjoin(gdf_geom_4326, gdf_store_circles.to_crs(4326), how="inner", predicate="intersects")
gdf_join['KFL_GPD'] = round(gdf_census_6993.to_crs(6933).geometry.area / 10**6, round_dec)
print(f'shape(gdf_join) = {gdf_join.shape[0]}')

gdf_int_dd = gdf_join.drop_duplicates(subset=['OBJID'])
gdf_int_dd['KFL_GPD'] = round(gdf_int_dd.to_crs(6933).geometry.area / 10**6, round_dec)
print(f'shape(gdf_int_dd) ={gdf_int_dd.shape[0]}')
### RELEVANT NUMBERS ###
int_gem = gdf_int_dd.shape[0]
int_ewz =gdf_join.EWZ.sum()
int_area_geom_nomerge = gdf_join.KFL_GPD.sum()
int_area_dbms_nomerge = gdf_join.KFL.sum()
int_area_geom_merge   = gdf_int_dd.KFL_GPD.sum()
int_area_dbms_merge   = gdf_int_dd.KFL.sum()

### MESSAGE ###
int_info_msg = f'# Towns={int_gem}, ' \
                f'# Area (geom, no merge)={int_area_geom_nomerge}Km2, ' \
                f'# Area (dbms, no merge)={int_area_dbms_nomerge}Km2, ' \
                f'Area (geom, merge)={int_area_geom_merge}Km2,' \
                f'Area (dbms, merge)={int_area_dbms_merge}Km2' \
                f'Pop={int_ewz}'

### FEATURE GROUP ###
fg_join = folium.FeatureGroup(name=f'Join/COMBINED [{int_info_msg}')
gjson_store = f_maps.get_folium_geojson(gdf_join, 
                                  fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD'],
                                  aliases = ['Name', 'Population', 'Area_dbms', 'Area_Geom'])
gjson_store.add_to(fg_join)

shape(gdf_join) = 772
shape(gdf_int_dd) =538


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## 3.6. GADMs

### Feature Group Countries (`fg_circ_merged_country`)

In [99]:
gdf_world = f_maps.load_gdf_from_csv(path=paths.path_wca)
print(gdf_world.shape)
print(gdf_world.head(1))

(255, 6)
                                            geometry  ADMIN ISO_A3  \
0  POLYGON ((-69.99694 12.57758, -69.93639 12.531...  Aruba    ABW   

   Area_total_km2  Area_land_km2  world_land_perc  
0           180.0          180.0              0.0  


In [100]:
gdf_int_world = gpd.overlay(gdf_world.to_crs(6933), gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_int_world = gdf_int_world.drop_duplicates()
gdf_int_world['area_intersection'] = round(gdf_int_world.geometry.area / 10**6, round_dec)
# print(gdf_int_world.shape)
# print(gdf_int_world.head(1))

#### `gdf_world`

##### `fg_country_all` (gdf_world)

In [101]:
# Filtering relevant countries
gdf_world_contain = gdf_world[gdf_world['ISO_A3'].isin(gdf_int_world['ISO_A3'])]

# gdf_world_contain
fg_country_all = folium.FeatureGroup(name=f'Countries Intersecting [{gdf_world_contain.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_world_contain.to_crs(4326), 
                                  fields=['ADMIN', 'Area_total_km2'], 
                                  aliases=['name', 'Area [km2]'])
gjson_store.add_to(fg_country_all)

##### `fg_country_int` (gdf_world)

In [102]:
fg_country_int = folium.FeatureGroup(name=f'ALL Country Join [{gdf_int_world.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_int_world.to_crs(4326), 
                                  fields=['ADMIN','area_intersection'], 
                                  aliases=['name', 'Area [km2]'])
gjson_store.add_to(fg_country_int)

# fg_join_country = folium.FeatureGroup(name=f'Country Join [{gdf_int_world.shape[0]}]')
# for i,(index,country) in enumerate(gdf_int_world.iterrows()):
#     print(f'[{i}/{gdf_int_world.shape[0]}] Country={country.ADMIN}')
#     gpd_country = gpd.GeoDataFrame([country.to_dict()]).set_crs(6399).to_crs(4326)
#     gjson = f_maps.get_folium_geojson(gpd_country, fields=['ADMIN'], aliases=['name'])
#     gjson.add_to(fg_join_country)

#### GADM

In [103]:
path = 'assets/Geo/GADM/gadm41_'
country_3 = 'AUT'  # 3-letter country code
l_adm = [
    # 'ADM_ADM_0',    # COUNTRY
    'ADM_ADM_1',    # STATE
    # 'ADM_ADM_2',    # REGION
    # 'ADM_ADM_3',    # COUNTY
    # 'ADM_ADM_4',    # MUNICIPALITY
    # 'ADM_ADM_5',  # SUB-MUNICIPALITY (Only France, Rwanda), aka PARISH
    ]
ext  = 'gpkg'
path_adm = f'{path}{country_3}.{ext}'
print(f'path_adm={path_adm}')  # PATH where the file is stored
logging = True

path_adm=assets/Geo/GADM/gadm41_AUT.gpkg


In [104]:
gdf_world = f_maps.load_gdf_from_csv(path=paths.path_wca)
gdf_int_world = gpd.overlay(gdf_world.to_crs(6933), gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_int_world = gdf_int_world.drop_duplicates()


l_country_iso = list(gdf_int_world.ISO_A3.values)

##### `fg_country_all` (GADM=0)

In [105]:
l_adm = [
    'ADM_ADM_0',    # COUNTRY
    ]

In [106]:
for i,iso3 in enumerate(l_country_iso):
    print(f'[{i+1}/{len(l_country_iso)}] Country={iso3}')
    path_adm = f'{path}{iso3}.{ext}'
    [l_gdf, l_name_adm, l_tooltip_adm ] = f_maps.extract_gdf_gpkg(
        path       = path_adm, 
        l_levels   = l_adm,
        is_logging = False
    )
    if i==0:
        gdf_adm0 = l_gdf[0]
    else:
        gdf_adm0= pd.concat([gdf_adm0, l_gdf[0]])

[1/3] Country=CHE
path=assets/Geo/GADM/gadm41_CHE.gpkg
ADM=0
[2/3] Country=DEU
path=assets/Geo/GADM/gadm41_DEU.gpkg
ADM=0
[3/3] Country=FRA
path=assets/Geo/GADM/gadm41_FRA.gpkg
ADM=0


In [107]:
gdf_int_countries = gpd.overlay(gdf_adm0.to_crs(6933), gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_int_countries = gdf_int_countries.drop_duplicates()
gdf_int_countries['area_intersection'] = round(gdf_int_countries.geometry.area / 10**6, round_dec)
print(gdf_int_countries.shape)
print(gdf_int_countries.head(1))

(3, 7)
  GID_0      COUNTRY  num_stores      area  perc_area  \
0   CHE  Switzerland          14  14119.32       0.39   

                                            geometry  area_intersection  
0  MULTIPOLYGON (((722255.399 5400604.903, 722938...             783.41  


In [108]:
# Filtering relevant countries
col_filter = 'COUNTRY'
col_alias = 'Country'
gdf_adm0_map = gdf_adm0[gdf_adm0[col_filter].isin(gdf_int_countries[col_filter])]
print(gdf_adm0_map.shape)
gdf_adm0_map = gdf_adm0_map.drop_duplicates()
print(gdf_adm0_map.shape)

(3, 3)
(3, 3)


In [109]:
fg_gadm0_all = folium.FeatureGroup(name=f'Countries [{gdf_adm0_map.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_adm0_map,
                                  fields=['COUNTRY', col_filter], 
                                  aliases=['Country', col_alias])
gjson_store.add_to(fg_gadm0_all)    

##### `fg_country_int` (GADM=0)

In [110]:
gdf_gadm0_int = gpd.overlay(gdf_adm0_map.to_crs(6933), gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_gadm0_int = gdf_gadm0_int.drop_duplicates()
gdf_gadm0_int['area_intersection'] = round(gdf_gadm0_int.geometry.area / 10**6, round_dec)
# print(gdf_gadm0_int.shape)
# print(gdf_gadm0_int.head(1))

In [111]:
fg_gadm0_ind = folium.FeatureGroup(name=f'ALL Countries Join [{gdf_gadm0_int.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_gadm0_int.to_crs(4326), 
                                  fields=['COUNTRY', 'area_intersection'], 
                                  aliases=['Country', 'Area[km2]'])
gjson_store.add_to(fg_gadm0_ind)

##### `fg_states_all` (GADM=1)

In [112]:
l_adm = [
    'ADM_ADM_1',    # STATE
    ]

In [113]:


for i,iso3 in enumerate(l_country_iso):
    print(f'[{i+1}/{len(l_country_iso)}] Country={iso3}')
    path_adm = f'{path}{iso3}.{ext}'
    [l_gdf, l_name_adm, l_tooltip_adm ] = f_maps.extract_gdf_gpkg(
        path       = path_adm, 
        l_levels   = l_adm,
        is_logging = False
    )
    if i==0:
        gdf_adm1 = l_gdf[0]
    else:
        gdf_adm1 = pd.concat([gdf_adm1, l_gdf[0]])

# print(gdf_adm1.shape)
# print(gdf_adm1.head(1))

[1/3] Country=CHE
path=assets/Geo/GADM/gadm41_CHE.gpkg
ADM=1
yes
l_columns=['COUNTRY', 'GID_0', 'NAME_1', 'TYPE_1']
[2/3] Country=DEU
path=assets/Geo/GADM/gadm41_DEU.gpkg
ADM=1
yes
l_columns=['COUNTRY', 'GID_0', 'NAME_1', 'TYPE_1']
[3/3] Country=FRA
path=assets/Geo/GADM/gadm41_FRA.gpkg
ADM=1
yes
l_columns=['COUNTRY', 'GID_0', 'NAME_1', 'TYPE_1']


In [114]:
gdf_int_states = gpd.overlay(gdf_adm1.to_crs(6933), gdf_store_circles_merged.to_crs(6933), how='intersection')
gdf_int_states = gdf_int_states.drop_duplicates()
gdf_int_states['area_intersection'] = round(gdf_int_states.geometry.area / 10**6, round_dec)
# print(gdf_int_states.shape)
# print(gdf_int_states.head(1))

In [115]:
# Filtering relevant countries
col_filter = 'NAME_1'
col_alias = 'State'
gdf_adm1_map = gdf_adm1[gdf_adm1[col_filter].isin(gdf_int_states[col_filter])]
print(gdf_adm1_map.shape)
gdf_adm1_map = gdf_adm1_map.drop_duplicates()
print(gdf_adm1_map.shape)

(12, 12)
(12, 12)


In [116]:
fg_states_all = folium.FeatureGroup(name=f'States [{gdf_adm1_map.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_adm1_map,
                                  fields=['COUNTRY', col_filter], 
                                  aliases=['Country', col_alias])
gjson_store.add_to(fg_states_all)

##### `fg_states_int` (GADM=1)

In [117]:
fg_states_int = folium.FeatureGroup(name=f'ALL States Join [{gdf_int_states.shape[0]}]')
gjson_store = f_maps.get_folium_geojson(gdf_int_states.to_crs(4326), 
                                  fields=['COUNTRY', 'NAME_1', 'area_intersection'], 
                                  aliases=['Country', 'State', 'Area[km2]'])
gjson_store.add_to(fg_states_int)

### Feature Group JOIN (`fg_join` using [`gdf_geom_4326` and `gdf_overlay`]

In [119]:
# gdf_geom_4326.head(1)

############# JOIN ####################
gdf_join = gpd.sjoin(gdf_geom_4326, gdf_store_circles.to_crs(4326), how="inner", predicate="intersects")
gdf_join['KFL_GPD'] = round(gdf_census_6993.to_crs(6933).geometry.area / 10**6, round_dec)
print(f'shape(gdf_join) = {gdf_join.shape[0]}')

gdf_int_dd = gdf_join.drop_duplicates(subset=['OBJID'])
gdf_int_dd['KFL_GPD'] = round(gdf_int_dd.to_crs(6933).geometry.area / 10**6, round_dec)
print(f'shape(gdf_int_dd) ={gdf_int_dd.shape[0]}')
### RELEVANT NUMBERS ###
int_gem = gdf_int_dd.shape[0]
int_ewz =gdf_join.EWZ.sum()
int_area_geom_nomerge = gdf_join.KFL_GPD.sum()
int_area_dbms_nomerge = gdf_join.KFL.sum()
int_area_geom_merge   = gdf_int_dd.KFL_GPD.sum()
int_area_dbms_merge   = gdf_int_dd.KFL.sum()

### MESSAGE ###
int_info_msg = f'# Towns={int_gem}, ' \
                f'# Area (geom, no merge)={int_area_geom_nomerge}Km2, ' \
                f'# Area (dbms, no merge)={int_area_dbms_nomerge}Km2, ' \
                f'Area (geom, merge)={int_area_geom_merge}Km2,' \
                f'Area (dbms, merge)={int_area_dbms_merge}Km2' \
                f'Pop={int_ewz}'

### FEATURE GROUP ###
fg_join = folium.FeatureGroup(name=f'Join/COMBINED [{int_info_msg}')
gjson_store = f_maps.get_folium_geojson(gdf_join, 
                                  fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD'],
                                  aliases = ['Name', 'Population', 'Area_dbms', 'Area_Geom'])
gjson_store.add_to(fg_join)

shape(gdf_join) = 772
shape(gdf_int_dd) =538


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## 3.7. TOWN Markers

### TOWN MARKERS

In [123]:
l_center_points = list(zip(pdf_stores['lat'], pdf_stores['lon'], pdf_stores['store']+'_'+pdf_stores['name']))#_{pdf_stores['name']}'))
# l_center_points

# ################### Distance to point of interest ##########
gdf_dist = pd.DataFrame(gdf_intersection[['ARS','lat','lon',col_perc_geom,'GEN','EWZ']])
gdf_dist = gpd.GeoDataFrame(gdf_dist, crs='epsg:4326', 
                    geometry=[geometry.Point(xy) for xy in zip(gdf_dist['lon'], gdf_dist['lat'])])
print(f'Shape(gdf_dist)={gdf_dist.shape[0]}')
gdf_dist = gdf_dist.drop_duplicates()
print(f'Shape(gdf_dist)={gdf_dist.shape[0]}')
for i,store_coords in enumerate(l_center_points):
    print(f'[{i+1}/{len(l_center_points)}] store_coords={store_coords}')
    gdf_dist = f_maps.compute_dist_to_lat_lon(gdf_dist, col_out=f'dist_{store_coords[2]}', 
                                              lon=store_coords[1], lat=store_coords[0], 
                                              round_dec=round_dec, units='km')

l_dist_cols = [col for col in gdf_dist.columns if col.startswith('dist_')]
gdf_dist['dist'] = gdf_dist[l_dist_cols].min(axis=1)
gdf_dist['dist_min_store'] = gdf_dist[l_dist_cols].idxmin(axis=1)
gdf_dist['dist_min_store'] = gdf_dist['dist_min_store'].apply(lambda x: x.replace('dist_',''))



l_dist_to_remove = gdf_dist.columns[gdf_dist.columns.str.startswith('dist_')].tolist()
l_dist_to_remove.remove('dist_min_store')
# l_dist_to_remove
gdf_dist = gdf_dist.drop(columns=l_dist_to_remove)
# gdf_dist

Shape(gdf_dist)=538
Shape(gdf_dist)=538
[1/14] store_coords=(49.00488565, 8.449674128073184, 'decathlon_Karlsruhe-Durlach')
[2/14] store_coords=(48.4007813, 9.9583729, 'decathlon_Ulm')
[3/14] store_coords=(48.7026898, 9.437392619846309, 'decathlon_Plochingen')
[4/14] store_coords=(48.7965766, 8.19518157114592, 'decathlon_Baden - Baden')
[5/14] store_coords=(48.8201891, 10.045626348192611, 'decathlon_Aalen-Essingen')
[6/14] store_coords=(49.3941626, 8.57783239839136, 'decathlon_Schwetzingen')
[7/14] store_coords=(48.6882907, 9.0090554, 'decathlon_Böblingen')
[8/14] store_coords=(47.610891699999996, 7.661843945356753, 'decathlon_Lörrach')
[9/14] store_coords=(48.900043, 9.1916945, 'decathlon_Ludwigsburg')
[10/14] store_coords=(49.1460258, 9.2214379, 'decathlon_Heilbronn-K3')
[11/14] store_coords=(48.472267200000005, 7.944426365060242, 'decathlon_Offenburg')
[12/14] store_coords=(47.7599556, 8.8420214, 'decathlon_Singen')
[13/14] store_coords=(48.791258850000006, 9.182104646022687, 'decat

In [124]:
#######################  Gemeinde Markers  ##########################

################
###  INSIDE  ###
################
gdf_inside = gdf_dist[gdf_dist['dist'] <= radius_km]
num_inside_towns = gdf_inside.shape[0]

fg_inside_circle = f_maps.get_markers_polygon(gdf_inside,
                                        l_tooltip=['GEN','dist', 'dist_min_store'],
                                        col_lat='lat', col_lon='lon', 
                                        name=f'Markers inside {gdf_inside.shape[0]}', color='darkgreen')
pop_inside = round(gdf_inside.EWZ.sum()/10**6,round_dec)
fg_inside = folium.FeatureGroup(name=f'Markers - Inside [towns:{num_inside_towns}, \
                                Pop:{pop_inside}M]')
fg_inside_circle.add_to(fg_inside)


################
###  PARTLY  ###
################
gdf_partly = gdf_dist[(gdf_dist['dist'] > radius_km)  & (gdf_dist[col_perc_geom] != 1.0)]
num_partly_towns = gdf_partly.shape[0]

fg_partly_circle = f_maps.get_markers_polygon(gdf_partly,
                                        l_tooltip=['GEN','dist', 'dist_min_store'], 
                                        col_lat='lat', col_lon='lon', 
                                        name=f'Markers partly {gdf_partly.shape[0]}', color='orange')
pop_partly = round(gdf_partly.EWZ.sum()/10**6,round_dec)
fg_partly = folium.FeatureGroup(name=f'Markers - Partly [towns:{num_partly_towns}, \
                                Pop:{pop_partly}M]')
fg_partly_circle.add_to(fg_partly)


# #################
# ###  OUTSIDE  ###
# #################
gdf_outside = gdf_census_6993[~gdf_census_6993['ARS'].isin(gdf_dist['ARS'])]
num_outside_towns = gdf_outside.shape[0]

fg_outside_circle = f_maps.get_markers_polygon(gdf_outside,
                                        l_tooltip=['GEN'], 
                                        col_lat='lat', col_lon='lon', 
                                        name=f'Markers outside {gdf_outside.shape[0]}', 
                                        color='red')
pop_outside = round(gdf_outside.EWZ.sum()/10**6,round_dec)
fg_outside = folium.FeatureGroup(name=f'Markers - Outside [towns:{num_outside_towns}, \
                                Pop:{pop_outside}M]')
fg_outside_circle.add_to(fg_outside)

### VALIDATION ###
print(f'Inside ={ num_inside_towns}')
print(f'Partly ={num_partly_towns}')
print(f'Outside ={num_outside_towns}')
print(f'Inside+Partly={num_inside_towns+ num_partly_towns}')
print(f'Total intersection={gdf_dist.shape[0]}')

print(f'Inside+Partly+Outside={num_inside_towns+ num_partly_towns+num_outside_towns}')
print(f'Total Geom ={gdf_geom_4326.shape[0]}')

Inside =439
Partly =99
Outside =565
Inside+Partly=538
Total intersection=538
Inside+Partly+Outside=1103
Total Geom =1103


In [125]:
# fg_inside.layer_name = fg_inside.layer_name + f'[{num_inside_towns}/{gpd_zensus_filt.shape[0]}]'
# fg_partly.layer_name = fg_partly.layer_name + f'[{num_partly_towns}/{gpd_zensus_filt.shape[0]}]'
# fg_int.layer_name = fg_int.layer_name + f'[Area={sum_int_kfl}, Area_Geom={round(sum_int_geom,round_dec)}, EWZ={sum_int_ewz} ]'


# 4.VISUALIZATION

In [ ]:
## ALL COMMENTED ##

# l_fg = [
    
#     ### 3.1. GEOMETRIES (Region, Towns) ###
    
#     fg_geom_region,     ### GEOJSON / REGION OF INTEREST (RoI) ###          #Bayern,Ba-Wu+Decathlon: 0.7s
#     fg_geom_town,       ### GEOJSON / REGION BREAKDOWN (Towns) ###

#     ### 3.2. STORES (Markers, Circles) ###    
#     # fglc,               ### MARKERS / STORES IN RoI                         #Bayern,Ba-Wu+Decathlon: 0.0s
#     # fg_circ,            ### GEOJSON / Circles of radius_km around stores    #Bayern,Ba-Wu+Decathlon: 2.2s
    
#     ### 3.3. GEOM+STORES (Intersections,Merges) ###
#     fg_int_circ_geom,   #(Location, Storename)                                     #Bayern,Ba-Wu+Decathlon: 54.0s
#     fg_mer_circ_geom,   #[Store,City, geom_in, geom_tot, geom_perc, pop_in, pop_tot, pop_perc]  GeoJson of intersection [Gem,circles]               #Bayern,Ba-Wu+Decathlon: 4.6s
    
#     # ### GEOJSON / INDIVIDUAL, INTERSECTION CIRCLE-DBMS DATA [EWZ, AREA]  ###
#     # ## fg_join,      #GeoJson of intersection [Gem,circles]               #Bayern,Ba-Wu+Decathlon: 2:52.2s
    
#     # ### GEOJSON / INDIVIDUAL, MERGE CIRCLE-DBMS DATA [EWZ, AREA]  ###
#     # fg_circ_merged,    #GeoJson with Circles of radius_km around stores     #Bayern,Ba-Wu+Decathlon: 0.1s
#     # fg_merge_ind,  #GeoJson of merge [Gem,circles]                      #Bayern,Ba-Wu+Decathlon: 0.5s
#     # fg_mer_circ_geom,         #Bayern,Ba-Wu+Decathlon: 1.0s

#     # ### GEOJSON / MERGED DATA, MERGE CIRCLE-DBMS DATA [EWZ, AREA] ###
#     # fg_merge,        #GeoJson of merge [Gem,circles]                      #Bayern,Ba-Wu+Decathlon: 2.2s

#     # ### GEOJSON / COUNTRIES ###
#     # fg_country_all,  # Duration: 0.3s (gdf_world)
#     # fg_gadm0_all,    # Duration: 20.2s (GADM=0)

#     # fg_country_int,    # Duration: 0.1s
#     # fg_gadm0_ind,      # Duration: 2.7s (GADM=0)

#     # ## GEOJSON / STATES ###
#     # fg_states_all,   # Duration: 21.9s (GADM=1)
#     # fg_states_int,   # Duration: 3.9s  (GADM=1)

#     # ## MARKERS / TOWNS ###
#     # fg_inside,     #Markers with Gem inside intersection (Gem,circle)   #Bayern,Ba-Wu+Decathlon: 7.0s
#     # fg_partly,      #Markers with Gem partly intersection (Gem,circle)   #Bayern,Ba-Wu+Decathlon: 1.9s
#     # fg_outside,     #Markers with Gem partly intersection (Gem,circle)   #Bayern,Ba-Wu+Decathlon: 6.5s
#     ]



# fg1 = folium.FeatureGroup(name='g1', show=False)
# fg2 = folium.FeatureGroup(name='g2', show=False)
# fg3 = folium.FeatureGroup(name='g3')
# folium.Marker([40, 74]).add_to(fg1)
# folium.Marker([38, 72]).add_to(fg2)
# folium.Marker([40, 72]).add_to(fg3)
# m.add_child(fg1)
# m.add_child(fg2)
# m.add_child(fg3)

# fglc = folium.plugins.GroupedLayerControl(
#     groups={'Countries': [fg1, fg2,fg3]},
#     exclusive_groups=False,
#     collapsed=False,
# )


In [18]:
# #OLD##########
# ### OLD ###
# ###########

# import folium
# m=folium.Map()


# l_fg_old = [
#     fg_circ_store,
#     fg_circ_merged,
#     fg_join,
#     fg_join_ind,
#     fg_merge_ind,
#     fg_merge
# ]

# for fg in l_fg_old:
#     fg.add_to(m)


# folium.LayerControl(position='topright',collapsed=False).add_to(m)
# print('Adding Legend...')
# m.fit_bounds(m.get_bounds())
# print('Adding fullscreen')

# folium.plugins.Fullscreen(position='topleft',
#                         title='Expand me',
#                         title_cancel='Exit me',
#                         force_separate_button=True).add_to(m)

# # m

In [19]:
# ## NEW ##

# import folium
# m=folium.Map()

# ##########
# ## NEW ###
# ##########


# l_l_fg_children = [
#     l_fg_geometry,
#     l_fg_merged_allstores
# # # #     # l_fg_stores
# # # #     l_fg_store
# ]
 

# l_fglc = [
#     fglc_geom, 
#     fglc_all_stores     
# #     # fglc_stores,    
# #     fglc_store
# ]

# # 3.1. GEOMETRIES (Region, Towns)   
# for l_fg in l_l_fg_children:
#     for fg in l_fg:
#         fg.add_to(m)
# for fglc in l_fglc:
#     fglc.add_to(m)

# # 3.2. STORES (Markers, Circles)    ### [fglc, fg_circ]
# for k,v in d_fglc.items():
#     print(f' Store={k}')
#     for fg in v['l_children']:
#         fg.add_to(m)
#     v['fglc'].add_to(m)


# ##########
# ## NEW ###
# ##########


# # for fg in l_fg:
# #     if type(fg) is list:
# #         for ffg in fg:
# #             ffg.add_to(m)
# #     else:
# #         fg.add_to(m)


# folium.LayerControl(position='topright',collapsed=False).add_to(m)
# print('Adding Legend...')
# m.fit_bounds(m.get_bounds())
# print('Adding fullscreen')

# folium.plugins.Fullscreen(position='topleft',
#                         title='Expand me',
#                         title_cancel='Exit me',
#                         force_separate_button=True).add_to(m)

# m

In [36]:
m.save('./bw_decathlon.html')

In [17]:
# m = folium.Map([40., 70.], zoom_start=6)

# fg1 = folium.FeatureGroup(name='g1', show=False)
# fg2 = folium.FeatureGroup(name='g2', show=False)
# fg3 = folium.FeatureGroup(name='g3')
# folium.Marker([40, 74]).add_to(fg1)
# folium.Marker([38, 72]).add_to(fg2)
# folium.Marker([40, 72]).add_to(fg3)
# m.add_child(fg1)
# m.add_child(fg2)
# m.add_child(fg3)

# folium.LayerControl(collapsed=False).add_to(m)

# folium.plugins.GroupedLayerControl(
#     groups={'groups1': [fg1, fg2]},
#     exclusive_groups=False,
#     collapsed=False,
# ).add_to(m)

# # m

In [529]:
gdf_merge_concat.shape

(153, 15)

In [531]:
gdf_merge_concat.to_file('./gdf_sued_germany2.json')

# `MONSTER CODE`

In [ ]:
gdf_intersection = gdf_store_circles.copy().set_crs(epsg=4326)
gdf_circle_6933 = gdf_intersection.to_crs(epsg=6933)

In [ ]:

l_fg = []
num_inside_towns, num_partly_towns = 0, 0
sum_ewz,sum_kfl, sum_area_geom_int = 0, 0, 0
sum_int_kfl, sum_int_ewz, sum_int_geom = 0, 0, 0


# l_fg_point, l_fg_circle, l_fg_merged = []
# l_fg_inside, l_fg_partly = []


fg_circle = folium.FeatureGroup(name=f'Circles [r={radius_km} Km, Area={round(3.1415 * radius_km**2, 2)} Km2]')
fg_join = folium.FeatureGroup(name=f'Intersections')
fg_inside = folium.FeatureGroup(name=f'Markers - Inside towns')
fg_partly = folium.FeatureGroup(name=f'Markers - Partly towns')
fg_outside = folium.FeatureGroup(name=f'Markers - Outside towns')
fg_overlay = folium.FeatureGroup(name=f'Overlay - Summary inside')

l_gdf_circles = []
l_gdf_merged_circles = []


for i,center_point in enumerate(l_center_points):
    lon = center_point[1]
    lat = center_point[0]
    geom_center_point = geometry.Point(lon, lat)

    # Convert the Shapely circle to a geoDataFrame
    poly_circle = f_maps.circle_around_lat_lon_point(
        lon=lon, lat=lat, radius=1000 * radius_km)
    gdf = gpd.GeoSeries([poly_circle])
    gdf_intersection = gdf.to_frame(name='geometry').set_crs(epsg=4326)
    gdf_circle_6933 = gdf_intersection.to_crs(epsg=6933)
    l_gdf_circles.append(gdf_circle_6933.iloc[0].geometry)

    # Generate the FeatureGroup and add it to the list
    # fg_circle = folium.FeatureGroup(name=f'Circle [lat={lat}, lon ={lon}, r={radius_km} Km]')
    gjson_circle = folium.GeoJson(gdf_intersection, color='red')
    gjson_circle.add_to(fg_circle)
    # l_fg_circle.append(fg_circle)

    ############# INTERSECTION ####################
    gdf_join = gpd.sjoin(gdf_geom_4326, gdf_intersection, 
                        how="inner", predicate="intersects") #\
                            # .drop(l_cols_non_serializable, axis=1)
                            # 

    gdf_join['KFL_GPD'] = round(gdf_census_6993.geometry.area / 10**6, 2)
    int_kfl = gdf_join.KFL.sum()
    int_ewz =gdf_join.EWZ.sum()
    int_geom = round(gdf_join.KFL_GPD.sum(), 2)
    sum_int_kfl += int_kfl
    sum_int_ewz += int_ewz
    sum_int_geom += int_geom

    print(f'int_kfl={int_kfl}Km2, int_geom={int_geom}Km2, int_ewz={int_ewz}')

    gjson_store = f_maps.get_folium_geojson(gdf_join, 
                                    fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD'],
                                        aliases = ['Name', 'Population', 'Area_KFL', 'Area_Geom'])
    gjson_store.add_to(fg_join)

    ############# OVERLAY / individual ####################

    col_perc = 'PERC_int'

    # Set the column used for NORMALIZATION (denominator)
    gdf_census_6993['area_geom'] = round(gdf_census_6993['geometry'].area / 10**6, 2)
    # Overlay geometry with polygon  
    gdf_intersection = gpd.overlay(gdf_census_6993, gdf_circle_6933, how='intersection')

    # Set the column used for NORMALIZATION (numerator)
    gdf_intersection['area'] = round(gdf_intersection['geometry'].area / 10**6, 2)
    gdf_intersection[col_perc] = gdf_intersection['area'] / gdf_intersection['area_geom']
    
    
    d_columns = {
        'area_geom':{'alias' : 'area',        'is_norm': True,  'is_int':False},
        'KFL':     {'alias' : 'area_data',    'is_norm': True,  'is_int':False},
        'EWZ':     {'alias' : 'Population',   'is_norm': True,  'is_int':True}
    }
    
    gdf_intersection = f_maps.overlay_shapes(gdf_circle=gdf_intersection, 
                                        col_perc=col_perc, 
                                        d_columns=d_columns)


    # ################### Distance to point of interest ##########
    gdf = pd.DataFrame(gdf_intersection[['ARS','lat','lon','PERC_int','GEN']])
    gdf = gpd.GeoDataFrame(gdf, crs='epsg:4326', 
                        geometry=[geometry.Point(xy) for xy in zip(gdf['lon'], gdf['lat'])])
    gdf = f_maps.compute_dist_to_lat_lon(gdf, lon=geom_center_point.x, lat=geom_center_point.y, round_dec=2, units='km')

    #######################  Gemeinde Markers  ##########################
    
    gdf_inside = gdf[gdf[col_perc] == 1.0]
    num_inside_towns += gdf_inside.shape[0]
    print(f'Inside ={ gdf_inside.shape[0]}')
    fg_inside_circle = f_maps.get_markers_polygon(gdf_inside,
                                            l_tooltip=['GEN','dist'], 
                                            name=f'Markers inside {gdf_inside.shape[0]}', color='darkgreen')
    fg_inside_circle.add_to(fg_inside)

    gdf_partly = gdf[gdf[col_perc] != 1.0]
    num_partly_towns += gdf_partly.shape[0]
    fg_partly_circle = f_maps.get_markers_polygon(gdf_partly, 
                                            l_tooltip=['GEN','dist'], 
                                            name=f'Markers partly {gdf_partly.shape[0]}', color='orange')
    fg_partly_circle.add_to(fg_partly)


    # gdf_outside = gdf[gdf[col_perc] != 1.0]
    
    ############# OVERLAY / merged ####################

    l_cols_percentaged = ['area_geom', 'EWZ', 'KFL']
    gdf_merge = f_maps.merge_shapes(gdf_intersection, l_col_percs=l_cols_percentaged,
                                    d_percs={'data':'KFL', 'geom':'area_geom'}, 
                                    num_dec=round_dec, is_logging=True)

    gdf_merge['num_in'] = gdf_inside.shape[0]
    gdf_merge['num_part'] = gdf_partly.shape[0]

    l_gdf_merged_circles.append(gdf_merge)
    
    gjson_store = f_maps.get_folium_geojson(
        gdf_merge, 
        fields=[
            'perc_data', 
            'KFL_int', 'KFL',
            'perc_geom',
            'area_geom_int', 'area_geom',
            'EWZ_int', 'EWZ',
            'num_in', 'num_part', 
            ],
        aliases = [
            'Percentage data', 
            'Area_data intersection [Km2]', 'Area_data shape [Km2]',
            'Percentage geometry',
            'Area_geom intersection [Km2]', 'Area_geom shape [Km2]',
            'Population intersection', 'Population shape',
            '# Localities inside', 
            '# Localities partially inside',
            ])
    gjson_store.add_to(fg_overlay)
    sum_ewz += gdf_merge['EWZ_int'].sum()
    sum_kfl += gdf_merge['KFL_int'].sum()
    sum_area_geom_int += gdf_merge['area_geom_int'].sum()
    print(f'area_geom_int= {sum_area_geom_int}\n')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


int_kfl=0.0Km2, int_geom=47661.49Km2, int_ewz=0.0
Inside =0
area_geom = 0.0
area_geom_int = 0.0
EWZ = 0
EWZ_int = 0
KFL = 0
KFL_int = 0.0
perc_data = nan
perc_geom = nan
area_geom_int= 0.0



c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


int_kfl=0.0Km2, int_geom=47661.49Km2, int_ewz=0.0
Inside =0
area_geom = 0.0
area_geom_int = 0.0
EWZ = 0
EWZ_int = 0
KFL = 0
KFL_int = 0.0
perc_data = nan
perc_geom = nan
area_geom_int= 0.0

int_kfl=0.0Km2, int_geom=47661.49Km2, int_ewz=0.0


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


Inside =0
area_geom = 0.0
area_geom_int = 0.0
EWZ = 0
EWZ_int = 0
KFL = 0
KFL_int = 0.0
perc_data = nan
perc_geom = nan
area_geom_int= 0.0

int_kfl=0.0Km2, int_geom=47661.49Km2, int_ewz=0.0


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


Inside =0
area_geom = 0.0
area_geom_int = 0.0
EWZ = 0
EWZ_int = 0
KFL = 0
KFL_int = 0.0
perc_data = nan
perc_geom = nan
area_geom_int= 0.0



In [58]:
gdf_intersection.columns

Index(['OBJID', 'ADE', 'GF', 'BSG', 'ARS', 'AGS', 'SDV_ARS', 'GEN', 'BEZ',
       'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2', 'SN_G',
       'FK_S3', 'NUTS', 'ARS_0', 'AGS_0', 'EWZ', 'KFL', 'DLM_ID', 'EPK',
       'EPK_norm', 'KFL_GPD', 'centroid', 'lat_1', 'lon_1', 'area_geom', 'url',
       'name', 'active', 'address', 'location', 'lat_2', 'lon_2', 'rect',
       'marker', 'fg', 'point', 'GADM_1', 'GADM_2', 'GADM_3', 'GADM_4',
       'store', 'geometry', 'area', 'PERC_int', 'area_geom_int', 'KFL_int',
       'EWZ_int'],
      dtype='object')

In [ ]:

fg_inside.layer_name = fg_inside.layer_name + f'[{num_inside_towns}/{gpd_census_filt.shape[0]}]'
fg_partly.layer_name = fg_partly.layer_name + f'[{num_partly_towns}/{gpd_census_filt.shape[0]}]'
fg_join.layer_name = fg_join.layer_name + f'[Area={sum_int_kfl}, Area_Geom={round(sum_int_geom,round_dec)}, EWZ={sum_int_ewz} ]'



# area_merged_circ = round(poly_merged_circ.area / 10**6, 3)
fg_overlay.layer_name = fg_overlay.layer_name + f'[Area_geom_int={sum_area_geom_int}, \
                                                    , KFL_int={sum_kfl} \
                                                     EWZ_int={sum_ewz}]'
# fg_circle.layer_name = f'Circles [Radius={radius_km} Km, \
#                                     Area={round(3.1415 * radius_km**2, 2)} Km2]'

In [223]:
l_gdf_circles

[<POLYGON ((1043641.029 5796631.545, 1043493.281 5795243.413, 1043074.219 579...>,
 <POLYGON ((986564.389 5738670.009, 986419.309 5737259.366, 986007.277 573586...>,
 <POLYGON ((968020.724 5805490.762, 967872.554 5804106.124, 967452.38 5802734...>,
 <POLYGON ((967989.589 5805241.335, 967841.43 5803856.599, 967421.288 5802485...>]

In [ ]:
geoseries_merged_circ = gpd.GeoSeries(l_gdf_circles)
poly_merged_circ = geoseries_merged_circ.union_all()
# poly_merged_circ

geoseries_all_poly_merged = gpd.GeoSeries([poly_merged_circ])
geoseries_all_poly_merged

gdf_circ_poly_merged = geoseries_all_poly_merged.to_frame(name='geometry').set_crs(epsg=6933)
gdf_circ_poly_merged


# Set the column used for NORMALIZATION (denominator)
gdf_census_6993['area_geom'] = round(gdf_census_6993['geometry'].area / 10**6, 2)
# Overlay geometry with polygon  
gdf_overlay_indiv = gpd.overlay(gdf_census_6993, gdf_circ_poly_merged, how='intersection')

# Set the column used for NORMALIZATION (numerator)
gdf_overlay_indiv['area'] = round(gdf_overlay_indiv['geometry'].area / 10**6, 2)
gdf_overlay_indiv[col_perc] = gdf_overlay_indiv['area'] / gdf_overlay_indiv['area_geom']


d_columns = {
    'area_geom':{'alias' : 'area',        'is_norm': True,  'is_int':False},
    'KFL':     {'alias' : 'area_data',    'is_norm': True,  'is_int':False},
    'EWZ':     {'alias' : 'Population',   'is_norm': True,  'is_int':True}
}

col_perc = 'PERC_int'
gdf_overlay_all = f_maps.overlay_shapes(gdf_circle=gdf_overlay_indiv, 
                                    col_perc=col_perc, 
                                    d_columns=d_columns)
# }
gdf_overlay_all.shape

(28, 37)

In [30]:
pd.set_option('display.max_columns', None)
gdf_overlay_indiv.head(5)

,OBJID,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,NBD,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,EWZ,KFL,DLM_ID,EPK,EPK_norm,KFL_GPD,lat,lon,area_geom,geometry,area,PERC_int,area_geom_int,KFL_int,EWZ_int
0,DEBKGVG2000006NB,6,4,1,120610020020,12061020,120610020020,Bestensee,Gemeinde,62,--,ja,12,0,61,00,20,020,R,DE406,120610020020,12061020,8932,38,DEBKGDL20000E3MM,235.052632,0.0,37.58,52.229444,13.643768,37.58,"POLYGON ((1315765.576 5796186.923, 1315811.148...",37.58,1.000000,37.58,38.000000,8932
1,DEBKGVG2000006NC,6,4,1,120610112112,12061112,120610112112,Eichwalde,Gemeinde,62,--,ja,12,0,61,01,12,112,R,DE406,120610112112,12061112,6475,2,DEBKGDL20000DZOT,3237.500000,0.0,2.83,52.372860,13.623927,2.83,"POLYGON ((1315435.222 5805593.956, 1316236.958...",2.83,1.000000,2.83,2.000000,6475
2,DEBKGVG2000006ND,6,4,1,120610217217,12061217,120610217217,Heidesee,Gemeinde,62,--,ja,12,0,61,02,17,217,R,DE406,120610217217,12061217,7416,135,DEBKGDL20000E61F,54.933333,0.0,136.12,52.247037,13.769812,136.12,"POLYGON ((1331632.52 5800945.781, 1332180.624 ...",132.85,0.975977,132.85,131.756906,7237
3,DEBKGVG2000006NF,6,4,1,120610260260,12061260,120610260260,Königs Wusterhausen,Stadt,61,--,ja,12,0,61,02,60,260,R,DE406,120610260260,12061260,38929,96,DEBKGDL20000E3NO,405.510417,0.0,95.82,52.314088,13.677780,95.82,"POLYGON ((1323821.412 5806531.436, 1323934.628...",95.82,1.000000,95.82,96.000000,38929
4,DEBKGVG2000006NJ,6,4,1,120610332332,12061332,120610332332,Mittenwalde,Stadt,61,--,ja,12,0,61,03,32,332,R,DE406,120610332332,12061332,9854,99,DEBKGDL20000E5P2,99.535354,0.0,99.28,52.248860,13.550094,99.28,"POLYGON ((1303626.156 5801804.117, 1305605.291...",99.28,1.000000,99.28,99.000000,9854


In [ ]:
# mask1 = gdf_geom_4326['ARS'].isin(gdf_overlay_indiv['ARS'])
# gdf_markers_intersect = gdf_geom_4326[mask1]


# gdf_markers_intersect['lat'] = gdf_markers_intersect.geometry.centroid.y
# gdf_markers_intersect['lon'] = gdf_markers_intersect.geometry.centroid.x
# gdf_markers_intersect = gdf_markers_intersect.to_crs(epsg=4326)
# gdf_markers_intersect = gpd.GeoDataFrame(gdf_markers_intersect, crs='epsg:4326', 
#                         geometry=[geometry.Point(xy) for xy in zip(gdf_markers_intersect['lon'], gdf_markers_intersect['lat'])])

# gdf_markers_intersect = pd.DataFrame(gdf_markers_intersect[['ARS','lat','lon','GEN','geometry']])

# gdf_markers_intersect = f_maps.compute_dist_to_lat_lon(gdf_markers_intersect, lon=center_point.x, lat=center_point.y, round_dec=2, units='km')
# # gdf_partly = gdf[gdf[col_perc] != 1.0]
# # num_partly_towns += gdf_partly.shape[0]
# # fg_partly_circle = f_maps.get_markers_polygon(gdf_partly, 
# #                                             name=f'Markers Inside {gdf_partly.shape[0]}', 
#                                               l_tooltip=['GEN','dist'], 
# #                                             color='orange')
# # fg_partly_circle.add_to(fg_partly)
# gdf_markers_intersect.head(5)

In [485]:
mask1 = ~gdf_geom_4326['ARS'].isin(gdf_overlay_indiv['ARS'])
gdf_outside = gdf_geom_4326[mask1]


gdf_outside['lat'] = gdf_outside.geometry.centroid.y
gdf_outside['lon'] = gdf_outside.geometry.centroid.x
gdf_outside = gdf_outside.to_crs(epsg=4326)
gdf_outside = gpd.GeoDataFrame(gdf_outside, crs='epsg:4326', 
                        geometry=[geometry.Point(xy) for xy in zip(gdf_outside['lon'], gdf_outside['lat'])])
gdf_outside = pd.DataFrame(gdf_outside[['ARS','lat','lon','GEN','geometry']])

gdf = f_maps.compute_dist_to_lat_lon(gdf_outside, lon=geom_center_point.x, lat=geom_center_point.y, round_dec=2, units='km')

fg_outside = f_maps.get_markers_polygon(gdf_outside,
                                            name=f'Markers outside [{gdf_outside.shape[0]}/{gdf_geom_4326.shape[0]}]', 
                                            l_tooltip=['GEN','dist'], 
                                            color='red')

NameError: name 'gdf_overlay_indiv' is not defined

In [ ]:


############# OVERLAY / merged ####################

l_cols_percentaged = ['area_geom', 'EWZ', 'KFL']
gdf_merge = f_maps.merge_shapes(gdf_overlay_all, l_col_percs=l_cols_percentaged,
                                d_percs={'data':'KFL', 'geom':'area_geom'}, 
                                num_dec=round_dec, is_logging=True)

gdf_merge['num_in'] = gdf_inside.shape[0]
gdf_merge['num_part'] = gdf_partly.shape[0]

l_gdf_merged_circles.append(gdf_merge)


exploded_gdf = gdf_circ_poly_merged.explode()
fg_overlay_all = folium.FeatureGroup(name=f'Overlay all [elems={len(l_gdf_circles)}, merged_elems={len(exploded_gdf)}' + \
                                    f'KFL_int={gdf_merge.KFL_int.sum()} km2,' + \
                                    # 'KFL={gdf_merge.KFL.sum()},' + \
                                    f'Area_geom_int={gdf_merge.area_geom_int.sum()} km2,' + \
                                    # 'Area_Geom={gdf_merge.area_geom.sum()},' + \
                                    f'EWZ_int={gdf_merge.EWZ_int.sum()},' 
                                    # 'EWZ={gdf_merge.EWZ.sum()} ]' + \
                                    )
gjson_store = f_maps.get_folium_geojson(
    gdf_merge, 
    fields=[
        'perc_data', 
        'KFL_int', 'KFL',
        'perc_geom',
        'area_geom_int', 'area_geom',
        'EWZ_int', 'EWZ',
        'num_in', 'num_part', 
        ],
    aliases = [
        'Percentage data', 
        'Area_data intersection [Km2]', 'Area_data shape [Km2]',
        'Percentage geometry',
        'Area_geom intersection [Km2]', 'Area_geom shape [Km2]',
        'Population intersection', 'Population shape',
        '# Localities inside', 
        '# Localities partially inside',
        ])
gjson_store.add_to(fg_overlay_all)

area_geom = 1811.69
area_geom_int = 1024.78
EWZ = 329298
EWZ_int = 236771
KFL = 1798
KFL_int = 1015.57
perc_data = 0.56
perc_geom = 0.57


In [ ]:
# import folium.map

# l_fg = [
#     # fg_geom, 
#     fg_point, fg_circle, fg_join, 
#     fg_overlay, fg_overlay_all, 
#       fg_inside,fg_partly,
#      fg_outside
#     ]

# m=folium.Map()
# for fg in l_fg:
#     fg.add_to(m)
# folium.LayerControl(position='topright',collapsed=False).add_to(m)
# # m.add_child(folium.LatLngPopup())

# m.fit_bounds(m.get_bounds())
# m